Importing the nessesary libraries

In [ ]:
import geopandas as gpd
import numpy as np
from pulp import *

In [ ]:
# Load the shapefile
gdf = gpd.read_file("land_parcels.shp")

Find basic info

In [ ]:
print("\n\n-------------------------------INFO BELOW-------------------------------\n")

# Number of polygons
num_polygons = len(gdf)
print("NUMBER OF POLYGONS: {}".format(num_polygons))

# Carbon stores
carbon = gdf["carbon_sto"]
print("CARBON RANGES FROM {} TO {}.".format(carbon.min(), carbon.max()))

# Costs of polygons
costs = gdf["cost"]
print("COSTS RANGE FROM {} TO {}.".format(costs.min(), costs.max()))

Finding the average carbon value

In [ ]:
total = 0
for value in carbon:
    total += value

carbon_average = total / 100

print("AVERAGE CARBON VALUE: {}".format(carbon_average))

Reprojecting the shapefile and computing areas

In [ ]:
gdf = gdf.to_crs(epsg=3347)

# Compute areas
gdf["area_km2"] = gdf.geometry.area / 1e6 # I am dividing by 10^6 because 1 km^2 = 1 000 000 m^2 = 10^6 m^2

print("AREAS RANGE FROM {} TO {}".format(gdf["area_km2"].min(), gdf["area_km2"].max()))

Filtering outliers

In [ ]:
# filtering outliers
threshold = 0.1 # arbitrarily set minimum area to 0.1 km^2. Since the minimum in the dataset is greater than 30, nothing gets filtered out
filtered_gdf = gdf[gdf["area_km2"] >= threshold]

print("Polygons removed: {}".format(len(gdf) - len(filtered_gdf)))

Creating an adjacency matrix to keep track of adjacent polygons

In [ ]:
adjacency_matrix = np.zeros((len(filtered_gdf), len(filtered_gdf)), dtype=int)

# Check adjacency
for i, geom1 in enumerate(filtered_gdf.geometry):
        for j, geom2 in enumerate(filtered_gdf.geometry):
                    if i != j and geom1.touches(geom2):
                                    adjacency_matrix[i, j] = 1 # if 2 polygons are adjacent, set their corresponding matrix entry to 1

Storing important values of the filtered data

In [ ]:
filtered_gdf["area"] = filtered_gdf.geometry.area / 1e6

filtered_costs = filtered_gdf['cost']
filtered_carbon = filtered_gdf['carbon_sto']
filtered_areas = filtered_gdf['area']

sumof_costs = filtered_costs.sum()
sumof_areas = filtered_areas.sum()

We will now use Pulp to optimize the selection of polygons

In [ ]:
# create a dictionary to store the variables we want to optimize
# In this case, our variables are binary values representing whether or not to include it's corresponding polygon
parcel_vars = {i: LpVariable("x_{}".format(i), cat = "Binary") for i in range(len(filtered_gdf))}

# initialize problem, with argument to maximize
problem = LpProblem("Maximimze_the_carbon_storage", LpMaximize)

# this is what we need to maximize
problem += lpSum(parcel_vars[i] * filtered_carbon[i] for i in range(len(filtered_gdf)))

Introducing the constraints

In [ ]:
# constraint 1 is that budget <= 1/2 total cost
problem += lpSum(filtered_costs[i] * parcel_vars[i] for i in range(len(filtered_gdf))) <= 1/2 * sumof_costs

# constraint 2 is that no two selected polygons are adjacent
for i in range(len(filtered_gdf)):
    for j in range(len(filtered_gdf)):
        if adjacency_matrix[i, j] == 1:
            problem += parcel_vars[i] + parcel_vars[j] <= 1

# optional constraint 3 is that the total area of the selected polygons must be >= 1/4 of total area
problem += lpSum(filtered_areas[i] * parcel_vars[i] for i in range(len(filtered_gdf))) >= 1/4 * sumof_areas

Finally, call on pulp to solve the problem. Since it prints a lot of clutter, I have added some dividers.

In [ ]:
print("---------------------------------Problem optimizing algorithm below------------------------------------")
problem.solve()
print("---------------------------------Problem optimizing algorithm above------------------------------------")

Store the indexes of the selected polygons in a list

In [ ]:
selected_polygons = [i for i in range(len(filtered_gdf)) if parcel_vars[i].value() == 1]

This outputs the information about which polygons were selected

In [ ]:
print("{} polygons were selected. View their indexes (in filtered_gdf) below: ".format(len(selected_polygons)))
print(selected_polygons)

Checks if any polygons are adjacent for debugging purposes

In [ ]:
for i in selected_polygons:
    for j in selected_polygons:
        if adjacency_matrix[i, j] == 1:

            print(adjacency_matrix[i, j])

Finally, it produces some final information about the solution set

In [ ]:
print("Their total carbon storage is {} compared to an overall carbon storage of {}".format(sum(filtered_carbon[i] for i in selected_polygons), filtered_carbon.sum()))

print("Their total cost is {} compared to a budget of {}".format(sum(filtered_costs[i] for i in selected_polygons), 1/2 * sumof_costs))

print("Their total area is {} compared to an overall area of {} (in km^2)".format(sum(filtered_areas[i] for i in selected_polygons), sumof_areas))

print("DONE")

Plotting the terrain

In [ ]:
import matplotlib.pyplot as plt

# Create a GeoDataFrame with selected polygons
selected_gdf = filtered_gdf.iloc[selected_polygons]

# Plot the entire filtered GeoDataFrame and then overlay the selected polygons
fig, ax = plt.subplots(figsize=(10, 10))
filtered_gdf.plot(ax=ax, color='lightgrey', edgecolor='black')  # Plot all polygons
selected_gdf.plot(ax=ax, color='green', edgecolor='black')  # Highlight selected polygons in green

plt.title('Selected Polygons') # set plot title

plt.savefig("selected_polygons_plot.png", bbox_inches='tight') # save the plot locally

plt.show() # display the plot, if possible